In [ ]:
import os
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import BatchNormalization
import tensorflow.keras.preprocessing.image 

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
path = "E:\deep learning project\inaturalist_12K"

In [ ]:
def get_data(path,augmentation=True):
    train_path=os.path.join(path,"train")
    test_path=os.path.join(path,"val")
    if augmentation==False:
        train_generator=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split=0.1)
    else:
        train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                          rotation_range=90,
                                          zoom_range=0.2,
                                          shear_range=0.2,
                                          validation_split=0.1,
                                          horizontal_flip=True)
    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    train_data = train_generator.flow_from_directory(
    directory=train_path,
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42)
    valid_data=train_generator.flow_from_directory(
    directory=train_path,
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42)
    test_data=test_generator.flow_from_directory(
    directory=test_path,
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42)
    return train_data,valid_data,test_data

In [ ]:
image_size=256
output_size=10

In [ ]:
import wandb

In [ ]:
wandb.login()

In [ ]:
from wandb.keras import WandbCallback
class CNN(object):
    def __init__(self,no_conv_layers,kernel_size,learning_rate,epochs,padding,filter_no_metric,dense_layer_size=32,activation_func='relu',
               no_filters=32,image_size=256,drop_out=0.2,output_size=10,
               augmentation= True,batch_normalization=True):
        self.kenel_size=kernel_size
        self.learning_rate=learning_rate
        self.image_size=image_size
        self.output_size=output_size
        self.augmentation=augmentation
        self.batch_normalization=batch_normalization
        self.no_conv_layers=no_conv_layers
        self.dense_layer_size=dense_layer_size
        self.drop_out=drop_out
        self.no_filters=no_filters
        self.epochs=epochs
        self.padding=padding
        self.filter_no_metric= filter_no_metric
        self.initialize(no_conv_layers,kernel_size,learning_rate,epochs,padding,filter_no_metric,dense_layer_size=32,activation_func='relu',
                    no_filters=32,image_size=256,drop_out=0.2,output_size=10,
                    augmentation= True,batch_normalization=True)
    def initialize(self,no_conv_layers,kernel_size,learning_rate,epochs,padding,filter_no_metric,dense_layer_size=32,activation_func='relu',
                 no_filters=32,image_size=256,drop_out=0.2,output_size=10,
                    augmentation= True,batch_normalization=True
                 ):
        self.model=Sequential()
        for i in range(0,no_conv_layers):
            if i==0:
                self.model.add(Conv2D(no_filters,kernel_size, input_shape=(image_size, image_size, 3),kernel_initializer = "he_uniform",padding = padding,
                           data_format="channels_last"))
            else:
                if filter_no_metric=="1":
                    self.model.add(Conv2D(no_filters,kernel_size,kernel_initializer = "he_uniform",padding =padding))
                elif filter_no_metric=="2":
                    self.model.add(Conv2D(no_filters*(2**i),kernel_size,kernel_initializer = "he_uniform",
                                          padding =padding))
                elif filter_no_metric=="1/2":
                    self.model.add(Conv2D(no_filters*(1/(2**i)),kernel_size,kernel_initializer = "he_uniform",
                                          padding =padding))

            self.model.add(Activation(activation_func))
            if batch_normalization==True:
                self.model.add(BatchNormalization())
            self.model.add(MaxPooling2D(pool_size=(2,2)))
        if batch_normalization==True:
            self.model.add(BatchNormalization())
        self.model.add(Flatten())
        self.model.add(Dense(dense_layer_size))
        self.model.add(Activation(activation_func))
        if batch_normalization==True:
            self.model.add(BatchNormalization())
        self.model.add(Dropout(drop_out))
        self.model.add(Dense(output_size))
        self.model.add(Activation("softmax"))




In [ ]:
import numpy as np
from tensorflow.keras.optimizers import Adam
np.random.seed(2)
class Runner(object):
    def __init__(self,inaturalist_data=True,augmentation=True):
        if inaturalist_data:
            self.initialize_inaturalist_data(augmentation)
        else:
            self.initialize_data()

    def initialize_inaturalist_data(self,augmentation=True):
        self.train_data,self.valid_data,self.test_data= get_data(path,augmentation)
    

    def initialize_data(self):
        raise NotImplementedError("Please implement this method if you need other dataset.")

    @staticmethod
    def get_activation_function(key):
        mapper = {
      #"sigmoid": Sigmoid,
      "relu": 'relu'
      #"tanh": Tanh
    }
        assert key in mapper
        return mapper[key]

def run_wandb():
    wandb.init(project="bayesian", entity="cs21m003_cs21d406")
    config = wandb.config
    wandb.run.name=f"e_{config.epochs}_bs_{config.batch_size}_kernel_size_{config.kernel_size}_filters_{config.no_filters}_ac_{config.act_func}_rate_{config.learning_rate}_aug_{config.augmentation}_BN_{config.batch_normalization}_drp_{config.drop_out}_pad_{config.padding}_dense_{config.dense_size}_metric_{config.filter_no_metric}_type_{config.type}"
    runner=Runner(True,config.augmentation)
    """
    params = {
    "epochs"        : config.epochs,
    "batch_size"    : config.batch_size,
    "kernel_size"   : config.kernel_size,
    "no_filters"    : config.no_filters,
    "act_func"      : config.act_func,
    "learning_rate" : config.learning_rate,
    "augmentation"  : config.augmentation,
    "batch_normalization"  : config.batch_normalization,
    "drop_out"  : config.drop_out,
    "padding": config.padding,
    "dense_size":config.dense_size,
    "filter_no_metric":config.filter_no_metric

    }
    """
  #no_conv_layers,filter_size,learning_rate,epochs,padding,filter_size_metric,dense_layer_size=32,activation_func='relu',
              # no_filters=32,image_size=256,drop_out=0.2,output_size=10,
               #augmentation= True,batch_normalization=True)
    model_1=CNN(5,config.kernel_size,config.learning_rate,config.epochs,config.padding,config.filter_no_metric,config.dense_size,config.act_func,
              config.no_filters,image_size,config.drop_out,output_size,config.augmentation,config.batch_normalization)
    model_1.model.compile(
    optimizer=Adam(config.learning_rate),  # Optimizer
    loss="categorical_crossentropy", metrics="categorical_accuracy")
  #train_data,valid_data,test_data=get_data(path,augmentation=True)
    model_1.model.fit(runner.train_data,epochs=config.epochs,batch_size=config.batch_size,validation_data=runner.valid_data,
          callbacks=[WandbCallback()])
    loss,accuracy=model_1.model.evaluate(runner.test_data,batch_size=config.batch_size)
    print(f'test accuracy:{accuracy}')
    wandb.log({"test accuracy":accuracy})

def do_hyperparameter_search_using_wandb():
    sweep_config = {
    "name": "bayesian sweep",
    "method": "bayes",
    "metric":{
      "name": "ValidationAccuracy",
      "goal": "maximize"
    },
    "parameters":{
      "type": {"values": ['bayesian']},
      "epochs": {"values": [5, 10]}, 
      "batch_size": {"values": [ 64]}, 
      "kernel_size": {"values": [(4,4), (5,5)]}, 
      "no_filters": {"values": [32,64,128]},
      "act_func": {"values": ['elu', 'relu', 'selu']}, 
      "learning_rate": {"values": [1e-3, 1e-4]}, 
      "augmentation": {"values": [True,False]} , 
      "batch_normalization": {"values": [True,False]},
      "drop_out": {"values": [0.5]},
      "padding": {"values": ['same','valid']},
      "dense_size": {"values": [64,128]},
      "filter_no_metric": {"values": ["1","2","1/2"]}}}
  
    sweep_id = wandb.sweep(sweep_config, project = "bayesian",entity='cs21m003_cs21d406')
    wandb.agent(sweep_id, function=run_wandb,count=20)


if __name__ == '__main__':
    do_hyperparameter_search_using_wandb() 